prerequisiti

https://interactivebrokers.github.io/tws-api/index.html

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from ibapi.client import *
from ibapi.wrapper import *
import numpy as np

candele giapponesi

In [ ]:
class candele_giapponesi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self) 
        self.data = []
        
    def nextValidId(self, orderId: int):
        
        mycontract = Contract()
        mycontract.symbol = "AAPL"
        mycontract.secType = "STK"
        mycontract.exchange = "SMART"
        mycontract.currency = "USD"        
        self.reqMarketDataType(3)    
   
        self.reqHistoricalData(orderId, mycontract, '20240110 15:59:00 US/Eastern', '1 D', '1 hour', 'TRADES', 0, 1, 0, [])

    def historicalData(self, reqId, bar):
        self.data.append([bar.date, bar.close])
        app.disconnect()

app = candele_giapponesi()
app.connect('127.0.0.1', 7497, 123)
app.run()

In [ ]:
df = pd.DataFrame(app.data, columns=['DateTime', 'Close'])
df

buy

In [ ]:
class buy(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)

    def nextValidId(self, orderId: OrderId):

        mycontract = Contract()
        mycontract.symbol = "AAPL"
        mycontract.secType = "STK"    
        mycontract.exchange = "SMART"
        mycontract.currency = "USD"

        self.reqContractDetails(orderId, mycontract)

    def contractDetails(self, reqId: int, contractDetails: ContractDetails):
        print(contractDetails.contract)

        myorder = Order()
        myorder.orderId = reqId
        myorder.action = "BUY"
        myorder.tif = "GTC"
        myorder.orderType = "LMT"
        myorder.lmtPrice = 1440.80
        myorder.totalQuantity = 1

        self.placeOrder(myorder.orderId, contractDetails.contract, myorder)


    def openOrder(self, orderId: OrderId, contract: Contract, order: Order, orderState: OrderState):
        print(f"openOrder. orderId: {orderId}, contract: {contract}, order: {order}")

    def orderStatus(self, orderId: OrderId, status: str, filled: Decimal, remaining: Decimal, avgFillPrice: float, permId: int, parentId: int, lastFillPrice: float, clientId: int, whyHeld: str, mktCapPrice: float):
        print(f"orderId: {orderId}, status: {status}, filled: {filled}, remaining: {remaining}, avgFillPrice: {avgFillPrice}, permId: {permId}, parentId: {parentId}, lastFillPrice: {lastFillPrice}, clientId: {clientId}, whyHeld: {whyHeld}, mktCapPrice: {mktCapPrice}")

    def execDetails(self, reqId: int, contract: Contract, execution: Execution):
        print(f"reqId: {reqId}, contract: {contract}, execution: {execution}")

app = buy()
app.connect("127.0.0.1", 7497, 123)
app.run()